# Initializations

In [1]:
import os
import tweepy
from dotenv import load_dotenv
# import KEYS
import datetime
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import os
import requests
from dash import dcc
from dash import html
from plotly.subplots import make_subplots

In [2]:
#Should be loaded
API_KEY=""
API_SECRET_KEY=""
ACCESS_TOKEN = ""
ACCESS_SECRET=""
BEARER_TOKEN=""


In [ ]:
client = tweepy.Client( bearer_token=BEARER_TOKEN,
                        consumer_key=API_KEY,
                        consumer_secret=API_SECRET_KEY,
                        access_token=ACCESS_TOKEN,
                        access_token_secret=ACCESS_SECRET)


# Functions

In [11]:

def count_query(query, date):
    TimeNow = "2022-12-28T23:59:59.000Z"
    result = []
    flag = True
    while(True):
        if flag == True:
            MahsaAmini_fa_count_0 = client.get_all_tweets_count(query, end_time=f"{TimeNow}", granularity='day')
            result += MahsaAmini_fa_count_0.data
            TimeNow = MahsaAmini_fa_count_0.data[0]['start']
        if pd.to_datetime(MahsaAmini_fa_count_0.data[0]['start']) - pd.to_datetime(date) < datetime.timedelta(31):
            try:
                MahsaAmini_fa_count_0 = client.get_all_tweets_count(query, end_time=f"{TimeNow}",start_time = date, granularity='day')
            except:
                break
            result += MahsaAmini_fa_count_0.data
            TimeNow = MahsaAmini_fa_count_0.data[0]['start']
            flag = False
        if pd.to_datetime(MahsaAmini_fa_count_0.data[0]['start']) < pd.to_datetime(date):
            break
    return result



def df_maker(twitter_list):
    df = pd.DataFrame(twitter_list)
    df['start'] = pd.to_datetime(df['start'], format = "%Y-%m-%dT%H:%M:%S%z").dt.strftime('%Y-%m-%d')
    df['end'] = pd.to_datetime(df['end'], format = "%Y-%m-%dT%H:%M:%S%z").dt.strftime('%Y-%m-%d')
    df[['start','end']] = df[['start','end']].apply(pd.to_datetime)
    df.sort_values(by=['start'],inplace=True)
    df = df.reset_index(drop=True)
    return df






In [8]:

def annotate(fig, df, annot_date, annot_text,yshift=0,xshift=0,fontsize=14,arrow=True):
    fig.add_annotation(x=pd.to_datetime(annot_date), y=df["MahsaAmini"].loc[df["Date"] == pd.to_datetime(annot_date)].values[0]+plotMarkerConstant,
        text=annot_text,
        showarrow=arrow,
        font = {
            'family':"B nazanin",
            'size' : fontsize
        },
        arrowhead=2,
        xshift=xshift,
        
        yshift=yshift,)



# fig.add_annotation(x=pd.to_datetime("2022-10-15"), y=df["MahsaAmini"].loc[df["Date"] == pd.to_datetime("20221015")].values[0]+plotMarkerConstant,
#     text=,
#     showarrow=True,
#     font = {
#         'family':"B nazanin",
#         'size' : 12
        
#     },
#     arrowhead=2)

# Hashtags

## MahsaAmini Hashtag

In [12]:
starting_date = "2022-09-13T00:00:00.000Z"
result = count_query("#مهسا_امینی",starting_date)
df = df_maker(result)
df

,end,start,tweet_count
0,2022-09-14,2022-09-13,0
1,2022-09-15,2022-09-14,6228
2,2022-09-16,2022-09-15,133995
3,2022-09-17,2022-09-16,489179
4,2022-09-18,2022-09-17,1108859
...,...,...,...
102,2022-12-25,2022-12-24,1246068
103,2022-12-26,2022-12-25,1031228
104,2022-12-27,2022-12-26,969262
105,2022-12-28,2022-12-27,988357


## Strikes Hashtag

In [13]:
# query = "-#مهسا_امینی lang:fa -is:reply"
query = "#اعتصابات_سراسری"
result = count_query(query, starting_date)
df_strikes = df_maker(result)
df_strikes


,end,start,tweet_count
0,2022-09-14,2022-09-13,108
1,2022-09-15,2022-09-14,61
2,2022-09-16,2022-09-15,91
3,2022-09-17,2022-09-16,373
4,2022-09-18,2022-09-17,2501
...,...,...,...
102,2022-12-25,2022-12-24,193836
103,2022-12-26,2022-12-25,142954
104,2022-12-27,2022-12-26,143484
105,2022-12-28,2022-12-27,107287


In [14]:
df['Strikes'] = df_strikes['tweet_count']
df.rename(columns = {'tweet_count':'MahsaAmini'}, inplace = True)
df.rename(columns = {'start':'Date'}, inplace = True)
df

,end,Date,MahsaAmini,Strikes
0,2022-09-14,2022-09-13,0,108
1,2022-09-15,2022-09-14,6228,61
2,2022-09-16,2022-09-15,133995,91
3,2022-09-17,2022-09-16,489179,373
4,2022-09-18,2022-09-17,1108859,2501
...,...,...,...,...
102,2022-12-25,2022-12-24,1246068,193836
103,2022-12-26,2022-12-25,1031228,142954
104,2022-12-27,2022-12-26,969262,143484
105,2022-12-28,2022-12-27,988357,107287


## Non-Mahsa

In [ ]:
query = "-#مهسا_امینی lang:fa -is:reply -has:hashtags"
result = count_query(query, starting_date)
df_nonMahsa = df_maker(result)
df_nonMahsa

In [27]:
df['NonMahsaAmini'] = df_nonMahsa['tweet_count']

## Matplotlib

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
fig, ax = plt.subplots(figsize=(20, 8))

# Add x-axis and y-axis
ax.plot(df['start'],
        df['tweet_count'],
        color='purple')
ax.plot(df_strikes['start'],
        df_strikes['tweet_count'],
        color='red')
ax.grid()
plt.setp(ax.get_xticklabels(), rotation=90)
ax.set(xlabel="Date",
       ylabel="Number of Tweets",
       title="Distrbution of #MahsaAmini (In persian) since the start of Iran revolution")

plt.axvline(x = pd.to_datetime("20221208"), color = 'g', label = "MahsaAmini arrested")



# Visualizing using Plotly

In [21]:
plotMarkerConstant=1e5
fig = px.line(df, x="Date", y=["MahsaAmini","Strikes"],)
fig.update_layout(
    font_family = "sans-serif",
    title={
        'text': "Trend of hashtags in Iran Revolution 2022",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},

    xaxis_title="Date",
    yaxis_title="Number of Tweets",
    legend_title="Hashtags")


annotate(fig,df,"2022-10-15","<b>: 23 مهر<br> آتش سوزی زندان اوین <b>")
annotate(fig,df,"2022-09-13","<b>: 22 شهریور<br> دستگیری مهسا امینی <b>")
annotate(fig,df,"2022-09-16","<b>: 25 شهریور<br> کشته شدن مهسا امینی <b>",yshift=30,xshift=10)
annotate(fig,df,"2022-09-23","<b>: 1 مهر<br> آغاز تظاهرات سراسری <br> اشنویه <b>")
annotate(fig,df,"2022-09-30","<b>: 7 مهر<br> جمعه خونین <b>")
annotate(fig,df,"2022-10-02","<b>: 10 مهر<br> حمله به دانشگاه شریف <b>",yshift=-5,xshift=0,fontsize=12)
annotate(fig,df,"2022-10-22","<b>: 30 مهر<br> تظاهرات برلین <b>", fontsize=12)
annotate(fig,df,"2022-10-26","<b>: 4 آبان<br> چهلم مهسا امینی <br> شاهچراغ <b>",yshift=10)
annotate(fig,df,"2022-11-03","<b>: 12 آبان<br> چهلم حدیث نجفی<b>",yshift=10)
annotate(fig,df,"2022-11-15","<b>: 24 آبان<br> اعتصاب 24و25و26<b>",yshift=10)
annotate(fig,df,"2022-12-05","<b>: 14 آذر<br> اعتصاب 14و15و16<b>",yshift=12)
annotate(fig,df,"2022-12-08","<b>: 17 آذر<br> اعدام محسن شکاری<b>",arrow=False,yshift=0,fontsize=12)

# # fig.add_shape()
# fig.add_annotation(yref='paper',x=pd.to_datetime("20221230"), y=1.06, text="aaaa")

fig.show()



In [27]:


# fig.write_html("/home/arash/projects/IranProtest22.html")
# fig.write_image("/home/arash/projects/IranProtest22.png")
print(df['MahsaAmini'].sum())
print(df['Strikes'].sum())


531219729
113359861


## Dash

In [ ]:

fig = px.line(df, x="start", y="tweet_count")
# fig.show()

# fig.write_html("/home/arash/projects/IranProtest22.html")
app = dash.Dash(external_stylesheets=[dbc.themes.JOURNAL])
server = app.server

colors = {"background": "#FFFFFF", "text": "#1DA1F2"}

fig.update_layout(
    plot_bgcolor=colors["background"],
    paper_bgcolor=colors["background"],
    font_color=colors["text"],
)

app.layout = html.Div(
    style={"backgroundColor": colors["background"]},
    children=[
        html.H1(
            children="Tweets by Date",
            style={"textAlign": "center", "color": colors["text"]},
        ),
        html.Div(
            children="An example using Dash and the Twitter API recent search counts to see how much I've been Tweeting this week",
            style={"textAlign": "center", "color": colors["text"]},
        ),
        dcc.Graph(id="example-twitter", figure=fig),
    ],
)

# if __name__ == "__main__":
app.run(debug=True)

# Dollar Rate

In [19]:
dfs = pd.read_excel("dollar_rate.xlsx", sheet_name="Sheet1")
dfs.columns
df['Dollar_price'] = dfs['Rate']
df.tail()

,end,Date,MahsaAmini,Strikes,Dollar_price
102,2022-12-25,2022-12-24,1246068,193836,40580
103,2022-12-26,2022-12-25,1031228,142954,41000
104,2022-12-27,2022-12-26,969262,143484,41520
105,2022-12-28,2022-12-27,988357,107287,42200
106,2022-12-28,2022-12-28,1054566,134840,44000


In [22]:
import plotly.graph_objects as go
fig = make_subplots(specs=[[{"secondary_y": True}]])
plotMarkerConstant=1e5
fig.add_trace(
    go.Scatter(x=df["Date"], y=df["MahsaAmini"]),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = df["Date"], y=df["Dollar_price"]),
    secondary_y=True
)
fig.add_trace(
    go.Scatter(x = df["Date"], y=df["Strikes"]),
    secondary_y=False
)
fig.update_layout(
    font_family = "sans-serif",
    title={
        'text': "Trend of hashtags in Iran Revolution 2022",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},

    xaxis_title="Date",
    yaxis_title="Number of Tweets",
    legend_title="Hashtags")

# fig.add_annotation(x=pd.to_datetime("2022-10-15"), y=df["MahsaAmini"].loc[df["Date"] == pd.to_datetime("20221015")].values[0]+plotMarkerConstant,
#     text=,
#     showarrow=True,
#     font = {
        
#         'size' : 14
#     },
#     arrowhead=2)
annotate(fig,df,"2022-10-15","<b>: 23 مهر<br> آتش سوزی زندان اوین <b>")
annotate(fig,df,"2022-09-13","<b>: 22 شهریور<br> دستگیری مهسا امینی <b>")
annotate(fig,df,"2022-09-16","<b>: 25 شهریور<br> کشته شدن مهسا امینی <b>",yshift=30,xshift=10)
annotate(fig,df,"2022-09-23","<b>: 1 مهر<br> آغاز تظاهرات سراسری <br> اشنویه <b>")
annotate(fig,df,"2022-09-30","<b>: 7 مهر<br> جمعه خونین <b>")
annotate(fig,df,"2022-10-02","<b>: 10 مهر<br> حمله به دانشگاه شریف <b>",yshift=-5,xshift=0,fontsize=12)
annotate(fig,df,"2022-10-22","<b>: 30 مهر<br> تظاهرات برلین <b>", fontsize=12)
annotate(fig,df,"2022-10-26","<b>: 4 آبان<br> چهلم مهسا امینی <br> شاهچراغ <b>",yshift=10)
annotate(fig,df,"2022-11-03","<b>: 12 آبان<br> چهلم حدیث نجفی<b>",yshift=10)
annotate(fig,df,"2022-11-15","<b>: 24 آبان<br> اعتصاب 24و25و26<b>",yshift=10)
annotate(fig,df,"2022-12-05","<b>: 14 آذر<br> اعتصاب 14و15و16<b>",yshift=12)
annotate(fig,df,"2022-12-08","<b>: 17 آذر<br> اعدام محسن شکاری<b>",arrow=False,yshift=0,fontsize=12)

# # fig.add_shape()
# fig.add_annotation(yref='paper',x=pd.to_datetime("20221230"), y=1.06, text="aaaa")

fig.show()



## Exporting Dataset in JSON

In [25]:
df.to_csv(r'IranProtest.csv', index=False)

# Clusters

## User finding

In [145]:
# query = "from:Outfarsi"
outfarsi_tweets = client.search_all_tweets(query, max_results=10, tweet_fields="public_metrics")
df_out = pd.DataFrame(outfarsi_tweets.data)
# df_out.head()
type(outfarsi_tweets.data[0])
# outfarsi_tweets.data[0].__dir__()

['__module__',
 '__doc__',
 '__slots__',
 '__init__',
 '__len__',
 '__repr__',
 '__str__',
 'attachments',
 'author_id',
 'context_annotations',
 'conversation_id',
 'created_at',
 'data',
 'edit_controls',
 'edit_history_tweet_ids',
 'entities',
 'geo',
 'id',
 'in_reply_to_user_id',
 'lang',
 'non_public_metrics',
 'organic_metrics',
 'possibly_sensitive',
 'promoted_metrics',
 'public_metrics',
 'referenced_tweets',
 'reply_settings',
 'source',
 'text',
 'withheld',
 '__abstractmethods__',
 '_abc_impl',
 '__hash__',
 '__eq__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__',
 '__contains__',
 '__getattr__',
 '__getitem__',
 '__iter__',
 'get',
 'keys',
 'items',
 'values',
 '__reversed__']

In [147]:
outfarsi_tweets.data[0]

<Tweet id=1606727325453893632 text='https://t.co/Km0BHp7iqR'>

In [156]:
# temp = client.get_retweeters(id = 1606727325453893632)
temp.data[0].id
# client.__dir__()

739522040726720513